In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

In [2]:
import numpy as np
import pandas as pd

#### import the Comedians vector

In [17]:
Comedians_vector = {'ramy youssef': ([0.20512821, 0.33333333, 0.20512821, 0.25641026]),
'andy woodhull': ([0.22222222, 0.55555556, 0.11111111, 0.11111111]),
'amy schumer': ([0.31343284, 0.34328358, 0.13432836, 0.20895522]),
'aziz ansari': ([0.23728814, 0.30508475, 0.3220339 , 0.13559322]),
'bert kreischer': ([0.21428571, 0.3       , 0.25714286, 0.22857143]),
'bill burr': ([0.24427481, 0.24427481, 0.29007634, 0.22137405]),
'chris rock': ([0.25641026, 0.32051282, 0.19871795, 0.22435897]),
'dave chappelle': ([0.3963964 , 0.18468468, 0.15765766, 0.26126126]),
'eric andre': ([0.11764706, 0.29411765, 0.33333333, 0.25490196]),
'george lopez': ([0.28571429, 0.28571429, 0.07142857, 0.35714286]),
'hannah gadsby': ([0.26229508, 0.40983607, 0.14754098, 0.18032787]),
'hasan minhaj': ([0.08064516, 0.29032258, 0.25806452, 0.37096774]),
'iliza shlesinger': ([0.19858156, 0.31914894, 0.25531915, 0.22695035]),
'jack whitehall': ([0.09375 , 0.359375, 0.265625, 0.28125 ]),
'jerry seinfeld': ([0.28378378, 0.28378378, 0.18918919, 0.24324324]),
'jo koy': ([0.19047619, 0.32142857, 0.26190476, 0.22619048]),
'jim jefferies': ([0.15483871, 0.26451613, 0.39354839, 0.18709677]),
'jimmy o. yang': ([0.20754717, 0.28301887, 0.22641509, 0.28301887]),
'joe rogan': ([0.28169014, 0.35211268, 0.22535211, 0.14084507]),
'kenny sebastian': ([0.1 , 0.25, 0.3 , 0.35]),
'kevin hart': ([0.26136364, 0.45454545, 0.19318182, 0.09090909]),
'marc maron': ([0.21276596, 0.21276596, 0.31914894, 0.25531915]),
'mark normand': ([0.25531915, 0.36170213, 0.19148936, 0.19148936]),
'michelle wolf': ([0.17241379, 0.27586207, 0.32758621, 0.22413793]),
'michael mcintyre': ([0.2       , 0.35555556, 0.26666667, 0.17777778]),
'neal brennan': ([0.16393443, 0.31147541, 0.2295082 , 0.29508197]),
'patton oswalt': ([0.18181818, 0.36363636, 0.31818182, 0.13636364]),
'pete davidson': ([0.15517241, 0.31034483, 0.31034483, 0.22413793]),
'ronny chieng': ([0.125   , 0.375   , 0.328125, 0.171875]),
'roy wood jr.': ([0.19767442, 0.39534884, 0.23255814, 0.1744186 ]),
'russell peters': ([0.12280702, 0.33333333, 0.33333333, 0.21052632]),
'sebastian maniscalco': ([0.09375 , 0.359375, 0.265625, 0.28125 ]),
'tiffany haddish': ([0.16666667, 0.41666667, 0.08333333, 0.33333333]),
'tom segura': ([0.16197183, 0.32394366, 0.30985915, 0.20422535]),
'trevor noah': ([0.23076923, 0.44230769, 0.23076923, 0.09615385]),
'urzila carlson': ([0.19444444, 0.27777778, 0.27777778, 0.25      ]),
'vir das': ([0.33333333, 0.33333333, 0.11111111, 0.22222222]),
'whitney cummings': ([0.26190476, 0.27380952, 0.22619048, 0.23809524])}

In [4]:
Comedians_vector["ramy youssef"]

[0.20512821, 0.33333333, 0.20512821, 0.25641026]

In [59]:
#get the comedians type
comedians_type = {}
for key,value in Comedians_vector.items():
    for i in range(4):
        if max(value) == value[i]:
            comedians_type[key] = i

#### get user vector

In [42]:
#get the compplete user_comedian_matrix
user_comedian_matrix_completion = pd.read_csv('./matrix_comple_finish.csv')
user_comedian_matrix_completion = user_comedian_matrix_completion.rename(columns={"Unnamed: 0":"User"})
user_comedian_matrix_completion.set_index(["User"], inplace=True)

In [93]:
user_vector = {}
for i in range(len(user_comedian_matrix_completion)):
    temp_name = (user_comedian_matrix_completion.index)[i].lower()
    user_vector[temp_name] = [0,0,0,0]
    num_list = [0,0,0,0]
    for j in range(user_comedian_matrix_completion.shape[1]):
        temp_com_name = list(user_comedian_matrix_completion)[j].lower()
        if temp_com_name in Comedians_vector:
            temp_index = comedians_type[temp_com_name]
            user_vector[temp_name][temp_index] += user_comedian_matrix.iloc[i,j]
            num_list[temp_index] += 1
    for k in range(4):
        if  num_list[k] != 0:
            user_vector[temp_name][k] /= num_list[k]
user_vector

{'a b': [0.0, 0.007976190476190475, 0.0, 0.0],
 'adedotun akande': [0.0, 0.04635416666666666, 0.0, 0.0],
 'anton nym': [0.0, 0.028374565972222224, 0.0, 0.0],
 'lydia waweru-morgan': [0.0, 0.010449218750000001, 0.0, 0.0],
 'jennifer bates': [0.0, 0.048828125, 0.030820105820105817, 0.0],
 'r. a.': [0.0, 0.027777777777777776, 0.0, 0.0],
 'ethan hayden': [0.0, 0.012152777777777778, 0.0, 0.0],
 'dan lowe': [0.0, -0.002430555555555555, 0.0, 0.0],
 'sumukh': [0.0, 0.021875000000000002, 0.0, 0.0],
 'a': [0.0, 0.036458333333333336, 0.0, 0.1390625],
 'brainflash1': [0.0, 0.0010416666666666654, 0.0, 0.0],
 'babychaingang': [0.0, 0.0, 0.0, -0.125],
 'l k': [0.0, 0.0, -0.008994708994708993, 0.0],
 'wuft chan': [0.0, 0.012016369047619048, 0.0, 0.0],
 'mike bee': [0.0, 0.02291666666666667, 0.0, 0.0],
 'bonafide slacker': [0.0, 0.0, 0.011111111111111112, 0.0],
 'tommaso urbisci': [0.0, 0.017708333333333333, -0.005740740740740741, 0.0],
 'mike jones': [0.0, -0.005208333333333333, 0.0, 0.05],
 'j': [0.0

#### get the useful user_comedian_matrix

In [107]:
user_comedian_matrix = pd.read_pickle('./user_comedian_matrix_OriginalPolarity.pkl')
user_comedian_matrix.columns = map(str.lower,user_comedian_matrix.columns)
user_comedian_matrix.index = map(str.lower,user_comedian_matrix.index)
user_comedian_matrix

,ramy youssef,andy woodhull,amy schumer,arsenio hall,aziz ansari,bert kreischer,bill burr,chris d'elia,chris rock,dave chappelle,...,ronny chieng,roy wood jr.,russell peters,sebastian maniscalco,tiffany haddish,tom segura,trevor noah,urzila carlson,vir das,whitney cummings
a b,0.191429,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
adedotun akande,NaN,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4,NaN,-0.1625,NaN,NaN,NaN,NaN,NaN,NaN
anton nym,NaN,0.68099,NaN,NaN,NaN,0,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lydia waweru-morgan,NaN,-0.2375,NaN,0.65,NaN,0.488281,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jennifer bates,NaN,0.671875,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
camthedestroyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jay jay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.155,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN
cynthia copland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,NaN
comedy central stand-up\n\n\n\n \n comedy central stand-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.268182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5


#### get the train set and test set

In [108]:
train_set = pd.read_csv('./training_set.csv')
train_set = train_set[{"Name","Comedian"}]
test_set = pd.read_csv('./test_set.csv')
test_set = test_set[{"Name","Comedian"}]

In [110]:
train_data = pd.DataFrame(columns = ["perfer1","perfer2","perfer3","perfer4","style1","style2","style3","style4","polarity"])
test_data = pd.DataFrame(columns = ["perfer1","perfer2","perfer3","perfer4","style1","style2","style3","style4","polarity"])

In [133]:
for i in range(len(train_set)):
    temp_com_name = train_set.iloc[i,0].lower()
    temp_user_name = train_set.iloc[i,1].lower()
    if temp_com_name in Comedians_vector and temp_user_name in user_vector:
        temp_com_list = Comedians_vector[temp_com_name]
        temp_user_list = user_vector[temp_user_name]
        train_data = train_data.append({"perfer1":temp_user_list[0], "perfer2":temp_user_list[1],
                                        "perfer3": temp_user_list[2], "perfer4": temp_user_list[3], 
                                        "style1":temp_com_list[0],"style2":temp_com_list[1],
                                        "style3":temp_com_list[2],"style4":temp_com_list[3],
                                        "polarity":user_comedian_matrix.loc[temp_user_name,temp_com_name]},
                                       ignore_index=True)

In [132]:
for i in range(len(test_set)):
    temp_com_name = test_set.iloc[i,0].lower()
    temp_user_name = test_set.iloc[i,1].lower()
    if temp_com_name in Comedians_vector and temp_user_name in user_vector:
        temp_com_list = Comedians_vector[temp_com_name.lower()]
        temp_user_list = user_vector[temp_user_name.lower()]
        test_data = test_data.append({"perfer1":temp_user_list[0], "perfer2":temp_user_list[1],
                                        "perfer3": temp_user_list[2], "perfer4": temp_user_list[3], 
                                        "style1":temp_com_list[0],"style2":temp_com_list[1],
                                        "style3":temp_com_list[2],"style4":temp_com_list[3],
                                        "polarity":user_comedian_matrix.loc[temp_user_name,temp_com_name]},
                                       ignore_index=True)

In [120]:
test_data.loc[test_data['polarity'] >= 0,'polarity' ] = 1
test_data.loc[test_data['polarity'] < 0,'polarity' ] = -1
train_data.loc[train_data['polarity'] >= 0,'polarity' ] = 1
train_data.loc[train_data['polarity'] < 0,'polarity' ] = -1

In [124]:
X_train = train_data.iloc[:,0:8]
X_test = test_data.iloc[:,0:8]
y_train = train_data[["polarity"]]
y_test = test_data[["polarity"]]

In [125]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

C:\python\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


0.7638888888888888

In [131]:
clf = LogisticRegression(penalty='l2')
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

C:\python\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.7638888888888888

In [142]:
clf = RandomForestClassifier(n_estimators=8)
clf.fit(X_train, y_train.values.ravel())
clf.score(X_test,y_test)

0.6805555555555556

In [140]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train.values.ravel())
clf.score(X_test,y_test)

0.625

In [141]:
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train.values.ravel())
clf.score(X_test,y_test)

0.7638888888888888